In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
ud = pd.read_csv('userdata.csv')
od = pd.read_csv('orderdata.csv')
id = pd.read_csv('interactiondata.csv')

In [ ]:
od = od.rename(columns={'userId': 'userid'})
id = id.rename(columns={'UserId': 'userid'})

od['avg_bill'] = od['avg_bill'].dropna()
#od = od[od['category']!='alcohol']
od = od[od['avg_bill']>0]

#id = id[(id['catalogue_purchases_count_avg']>=0) & (id['website_purchases_count_avg']>=0) & (id['mobile_purchases_count_avg']>=0) & (id['mobile_visit_count_avg']>=0)]

id['last_order_dt'] = pd.to_datetime(id['last_order_dt'])
ud['join_datetime'] = pd.to_datetime(ud['join_datetime'])

id['last_order_dt'] = id['last_order_dt'].fillna(pd.Timestamp("20240619"))
ud['education'] = ud['education'].fillna('basic')

id[['catalogue_purchases_count_avg', 
                 'website_purchases_count_avg', 
                 'mobile_purchases_count_avg', 
                 'mobile_visit_count_avg']] = id[['catalogue_purchases_count_avg', 
                                                                'website_purchases_count_avg', 
                                                                'mobile_purchases_count_avg', 
                                                                'mobile_visit_count_avg']].clip(lower=0)

In [ ]:
orderdata_agg = od.pivot_table(index='userid', 
                                      columns='category', 
                                      values='avg_bill', 
                                      aggfunc='mean').reset_index().fillna(0)

orderdata_agg.columns = ['userid'] + [f'avg_bill_{col}' for col in orderdata_agg.columns[1:]]

mdf = ud.merge(id, on='userid', how='left')
mdf = mdf.merge(orderdata_agg, on='userid', how='left')

In [ ]:
mdf['purchases'] = mdf['discount_purchases_count_avg'] + mdf['catalogue_purchases_count_avg'] + mdf['store_purchases_count_avg'] + mdf['website_purchases_count_avg'] + mdf['mobile_purchases_count_avg']

In [ ]:
mdf['visits'] = mdf['website_visit_count_avg'] + mdf['mobile_visit_count_avg']

In [ ]:
mdf.columns[7]

In [ ]:
mdf['avg_bill_month'] = mdf['avg_bill_alcohol'] + mdf['avg_bill_fish'] + mdf['avg_bill_fruits'] + mdf['avg_bill_meat'] + mdf['avg_bill_other'] + mdf['avg_bill_sweets'] + mdf['avg_bill_vegetables']

In [ ]:
mdf['client_dur'] = mdf['last_order_dt'] - mdf['join_datetime']

In [ ]:
mdf['client_dur']

In [ ]:
mdf.drop(columns=['userid', 'marital', 'education'])

In [ ]:
plt.hist(mdf['avg_bill_month'], bins = 30)

In [ ]:
import plotly.express as px
#df = px.data.iris()
fig = px.scatter_3d(mdf, x='income', y='age', z='avg_bill_month',
              color='client_dur')
fig.show()

In [ ]:
mdf[mdf['age']<16]['education'].value_counts()

In [ ]:
plt.hist(mdf[mdf['age']<12]['avg_bill_month'], bins=30)

In [ ]:
mdf[mdf['age']==14]['income'].describe()

In [ ]:
plt.plot(mdf['income'], mdf['avg_bill_month'], 'ro')

In [ ]:
plt.matshow(mdf.drop(columns=['userid', 'marital', 'education']).corr())
plt.figure(figsize=(30,30), dpi=100)

In [ ]:
# Find who gives more money